In [4]:
import modulos as md
import numpy as np
symbol='SFPUSDT'

'''
lista=md.lista_de_monedas ()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_vwap(data)
        resultado = md.backtesting(data)
        if float(resultado[4]) > 1030:
            print(f"{symbol} - {resultado[4]}")
    except:
        pass        
'''
data = md.obtiene_historial(symbol)
data = md.estrategia_vwap(data)
resultado = md.backtesting(data)
print(resultado)

Start                     2023-06-05 04:30:00
End                       2023-06-26 00:00:00
Duration                     20 days 19:30:00
Exposure Time [%]                        42.3
Equity Final [$]                   1042.68835
Equity Peak [$]                   1061.831722
Return [%]                           4.268835
Buy & Hold Return [%]               13.341554
Return (Ann.) [%]                   100.07831
Volatility (Ann.) [%]               69.456878
Sharpe Ratio                          1.44087
Sortino Ratio                        5.876169
Calmar Ratio                         12.24751
Max. Drawdown [%]                   -8.171319
Avg. Drawdown [%]                   -1.101683
Max. Drawdown Duration        9 days 15:00:00
Avg. Drawdown Duration        1 days 00:20:00
# Trades                                   57
Win Rate [%]                        31.578947
Best Trade [%]                       5.233773
Worst Trade [%]                     -2.071352
Avg. Trade [%]                    